<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

# _**Creating Data Rows with Annotations with LabelSpark**_

<td>
<a href="https://github.com/Labelbox/labelspark/blob/master/notebooks/annotations.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

## _**Documentation**_

### **Data Rows**
_____________________

**Requirements:**

- A `row_data` column - This column must be URLs that point to the asset to-be-uploaded

- Either a `dataset_id` column or an input argument for `dataset_id`
  - If uploading to multiple datasets, provide a `dataset_id` column 
  - If uploading to one dataset, provide a `dataset_id` input argument
    - _This can still be a column if it's already in your CSV file_

**Recommended:**
- A `global_key` column
  - This column contains unique identifiers for your data rows
  - If none is provided, will default to your `row_data` column
- An `external_id` column
  - This column contains non-unique identifiers for your data rows
  - If none is provided, will default to your `global_key` column  

**Optional:**
- A `project_id` columm or an input argument for `project_id`
  - If batching to multiple projects, provide a `project_id` column
  - If batching to one project, provide a `project_id` input argument
    - _This can still be a column if it's already in your CSV file_

### **Annotations**
_____________________

*Note:*
*There must also be a `project_id` column, or an input argument for `project_id` when using LabelSpark to upload annotations*

*There must also be an `upload_method` provided when using LabelSpark*
  - *`upload_method` must be one of the following:*
    - *`"mal"` (uploads annotations as pre-labels)*
    - *`"import"` (uploads annotations as submitted labels)*
    

- For annotations, the column name must be `annotation` + `divider` + `annotation_type` + `divider` + `top_level_feature_name`
  - Example: `annotation///bbox///bbox_tool_name` where, in this case, the bounding tool name in your Labelbox ontology is "bbox_tool_name"
  - `annotation_type` must be one of the following:
    - `bbox`, `polygon`, `point`, `mask`, `line`, `named-entity`, `radio`, `checklist`, `text`
- Values for annotations must correspond with the following, per annotation type:

_____________________
_____________________

**Row-Level Formats for Tool Annotations**
- `bbox` (this example is two annotations)
```
[
        [[top, left, height, width], [nested_classification_name_paths]], 
        [[top, left, height, width], [nested_classification_name_paths]]
]
```
- `polygon` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `line` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `point` (this example is two annotations)
```
[
        [[x, y], [nested_classification_name_paths]], 
        [[x, y], [nested_classification_name_paths]]
]
```
- `mask` (this example is two annotations)
```
[
        [[URL, colorRGB], [nested_classification_name_paths]], 
        [[URL, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[numpy_array, colorRGB], [nested_classification_name_paths]], 
        [[numpy_array, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[png_bytes, None], [nested_classification_name_paths]], 
        [[png_bytes, None], [nested_classification_name_paths]]
]
```
- `named-entity` (this example is two annotations)
```
[
        [[start, end], [nested_classification_name_paths]], 
        [[start, end], [nested_classification_name_paths]]
]
```

**Row-Level Formats for Classification Annotations**
- `radio`, `checklist` and `text`
```
[[answer_name_paths]]
```
  - Note: the last string in a text name path is the text answer value itself
_____________________
_____________________

## _**Code**_

Install LabelSpark

In [0]:
%pip install labelspark -q

Python interpreter will be restarted.
Python interpreter will be restarted.

In [0]:
import labelspark as ls
# Imported to create an example ontology - not required for typical runs of LabelSpark
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option

In [0]:
table_name = "labelspark_annotations_demo" # Name to register test table under
csv_path = "https://raw.githubusercontent.com/Labelbox/labelspark/master/datasets/annotations.csv" # Path to your CSV file
api_key = ""

Load a CSV as a Spark Table

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('labelspark-demo').getOrCreate() # Create a Spark Session
df = pd.read_csv(csv_path) # Read CSV in as Pandas DataFrame
table = spark.createDataFrame(df) # Convert your Pandas DataFrame into a Spark DataFrame

Register your Spark Table (for this demo, we will only save as a temporary table)

In [0]:
tblList = spark.catalog.listTables()

if not any([x.name == table_name for x in tblList]):
  table.createOrReplaceTempView(table_name)
  print(f"Registered table: {table_name}")

Registered table: labelspark_full_demo

In [0]:
display(sqlContext.sql(f"select * from {table_name} LIMIT 5"))

global_key row_data external_id annotation///bbox///sample_bounding_box annotation///bbox///sample_nested_bounding_box annotation///polygon///sample_polygon annotation///polygon///sample_nested_polygon annotation///mask///sample_segmentation_mask annotation///point///sample_point annotation///line///sample_polyline annotation///radio///sample_radio_question annotation///checklist///sample_checklist_question annotation///radio///sample_nested_radio_question annotation///text///sample_free_text_question labelspark-annotations-test-gCbn5IeZtE92OaUbyl1ZjQ.jpg https://labelbox.s3-us-west-2.amazonaws.com/datasets/mapillary_traffic/images/gCbn5IeZtE92OaUbyl1ZjQ.jpg gCbn5IeZtE92OaUbyl1ZjQ.jpg [[[1853, 191, 213, 304], []], [[1828, 749, 154, 130], []]] [[[1813, 1066, 259, 285], ['sample_tool_sub_text_question///Test text']]] [[[[3363.98, 1180.19], [3205.616, 1349.865], [3397.915, 1549.706], [3563.82, 1393.227]], []]] [[[[1341.067, 2550.793], [1412.708, 2545.137], [1718.125, 3001.377], [1593.696, 2986.294]], ['sample_tool_sub_radio_question///sample_sub_radio_answer_1']]] [[['...', 'null'], []]] [[[1936.818, 2509.317], []], [[732.12, 2473.496], []]] [[[[1416.479, 1962.584], [2768.229, 2235.95], [3763.661, 2441.446], [4016.289, 2492.349]], []]] [['sample_radio_answer_1']] [['sample_checklist_answer_1']] [['sample_branch_radio_answer_1///sample_sub_radio_question///sample_sub_radio_answer_2']] null labelspark-annotations-test-1MnLIosQZmXH3T-iU-4mtQ.jpg https://labelbox.s3-us-west-2.amazonaws.com/datasets/mapillary_traffic/images/1MnLIosQZmXH3T-iU-4mtQ.jpg 1MnLIosQZmXH3T-iU-4mtQ.jpg [[[1463, 2155, 125, 200], []]] [[[1362, 938, 237, 198], []]] [[[[19.327, 2805.486], [701.995, 2515.586], [1786.783, 2801.746], [3039.9, 2956.983], [4000, 2912.095], [4000, 3000], [1294.888, 2988.778], [664.589, 2653.99], [0, 2927.057]], []]] [[[[1902.743, 1460.723], [1897.132, 1543.017], [1899.002, 1617.83], [1975.686, 1623.441], [1975.686, 1569.202], [1975.686, 1526.185], [1975.686, 1470.075], [1953.242, 1434.539]], []]] [[['iVBORw0KGgoAAAANSUhEUgAAD6AAAAu4CAAAAADQ3hg7AAAvqklEQVR4nOzcuVYbURRFQez//2c5gLUA8bCm7nenqtiBFJ27u4Xf3gAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

Create a project, dataset and ontology

In [0]:
client = ls.Client(lb_api_key=api_key)

In [0]:
project = client.lb_client.create_project(name="LabelSpark-annotations-demo")
dataset = client.lb_client.create_dataset(name="LabelSpark-annotations-demo")

Creating a project without specifying media_type through this method will soon no longer be supported.
Default createProject behavior will soon be adjusted to prefer batch projects. Pass in `queue_mode` parameter explicitly to opt-out for the time being.

In [0]:
ontology_builder = OntologyBuilder(
    classifications=[ 
        Classification( # Radio classification
            class_type=Classification.Type.RADIO, instructions="sample_radio_question", 
            options=[Option(value="sample_radio_answer_1"), Option(value="sample_radio_answer_2")]
        ),
        Classification( # Checklist classification
            class_type=Classification.Type.CHECKLIST, instructions="sample_checklist_question", 
            options=[Option(value="sample_checklist_answer_1"), Option(value="sample_checklist_answer_2")]
        ), 
        Classification( # Text classification
            class_type=Classification.Type.TEXT, instructions="sample_free_text_question"
        ),
        Classification( # Radio classification where one answer has a nested radio classification
            class_type=Classification.Type.RADIO, instructions="sample_nested_radio_question",
            options=[
                Option(
                    value="sample_branch_radio_answer_1", 
                    options=[
                        Classification(
                            class_type=Classification.Type.RADIO, instructions="sample_sub_radio_question", 
                            options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                        )
                    ]
                ), 
                Option(value="sample_leaf_radio_answer_2")
            ]
        )
    ],
    tools=[ # List of Tool objects
        Tool( # Bounding Box tool
            tool=Tool.Type.BBOX, name="sample_bounding_box"), 
        Tool( # Bounding Box tool with a nested text classification
            tool=Tool.Type.BBOX,  name="sample_nested_bounding_box",
            classifications=[
                Classification(class_type=Classification.Type.TEXT, instructions="sample_tool_sub_text_question"),]
        ),
        Tool( # Polygon tool
            tool=Tool.Type.POLYGON, name="sample_polygon"
        ),
        Tool( # Polygon tool with a nested radio classification
            tool=Tool.Type.POLYGON, name="sample_nested_polygon",
            classifications=[
                Classification(
                    class_type=Classification.Type.RADIO, instructions="sample_tool_sub_radio_question",
                    options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                ),
            ]            
        ),        
        Tool( # Segmentation mask tool given the name "mask"
            tool=Tool.Type.SEGMENTATION, name="sample_segmentation_mask"
        ),
 	      Tool( # Point tool given the name "point"
            tool=Tool.Type.POINT, name="sample_point"
        ), 
        Tool( # Polyline tool given the name "line"
            tool=Tool.Type.LINE, name="sample_polyline"
        )
    ]
)

ontology = client.lb_client.create_ontology("LabelSpark-annotations-demo", ontology_builder.asdict())

project.setup_editor(ontology)

/databricks/python/lib/python3.8/site-packages/labelbox/schema/ontology.py:169: UserWarning: When creating the Classification feature, please use “name” for the classification schema name, which will be used when creating annotation payload for Model-Assisted Labeling Import and Label Import. “instructions” is no longer supported to specify classification schema name.
 warnings.warn(msg)

Upload to Labelbox

In [0]:
results = client.create_data_rows_from_table(
    table = table,
    dataset_id = dataset.uid,
    project_id = project.uid,
    upload_method = "import", # Must be either "import" or "mal"
    skip_duplicates = False, # If True, will skip data rows where a global key is already in use
    mask_method = "png", # Input masks must be either "png", "url", or "array"
    verbose = True, # If True, prints information about code execution
)

Creating upload list - 3 rows in Pandas DataFrame
Beginning upload for Dataset with ID clf7a85og0nyf070d9vrp0bx4
Vetting global keys
Warning: Global keys in this upload are in use by active data rows, attempting to add the following suffix to affected data rows: '///1'
Global keys vetted
Beginning data row upload for Dataset with ID clf7a85og0nyf070d9vrp0bx4 - uploading 3 data rows
Batch #1: 3 data rows
Success: Upload batch number 1 successful
Upload complete - all data rows uploaded
Sending 3 data rows to project with ID clf7a84tt0a8007yg1xyybxbi
All data rows have been batched to the specified project(s)
Uploading annotations as submitted labels (Label Import)
Uploading 51 annotations for 3 data rows to project with ID clf7a84tt0a8007yg1xyybxbi
Success: upload batch number 1 complete